In [2]:
import cv2
import os
import glob
import mediapipe as mp

# =========================
# PATH CONFIG
# =========================
DATASET_ROOT = "dataset-2/samples"
WEBCAM_ROOT = "WEBCAM"
LIVE_OUT = os.path.join(WEBCAM_ROOT, "live")
NOT_LIVE_OUT = os.path.join(WEBCAM_ROOT, "not_live")

os.makedirs(LIVE_OUT, exist_ok=True)
os.makedirs(NOT_LIVE_OUT, exist_ok=True)

# =========================
# MEDIAPIPE SETUP
# =========================
mp_face = mp.solutions.face_detection
face_detector = mp_face.FaceDetection(
    model_selection=1,        # 0: short-range, 1: full-range
    min_detection_confidence=0.6
)

# =========================
# PARAMETERS
# =========================
FRAME_INTERVAL = 5            # lấy dày hơn
IMG_SIZE = 256
MAX_FRAMES_PER_VIDEO = 50     # cho phép nhiều hơn

# =========================
# PROCESS VIDEOS
# =========================
video_paths = glob.glob(f"{DATASET_ROOT}/**/live_video.*", recursive=True)
print(f"🔍 Found {len(video_paths)} live videos")

img_count = 0

for vid_path in video_paths:
    cap = cv2.VideoCapture(vid_path)
    frame_id = 0
    saved = 0
    video_name = os.path.basename(os.path.dirname(vid_path))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_id % FRAME_INTERVAL == 0:
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_detector.process(rgb)

            if results.detections:
                # lấy face có confidence cao nhất
                det = max(
                    results.detections,
                    key=lambda d: d.score[0]
                )

                h, w, _ = frame.shape
                bbox = det.location_data.relative_bounding_box

                x1 = int(bbox.xmin * w)
                y1 = int(bbox.ymin * h)
                bw = int(bbox.width * w)
                bh = int(bbox.height * h)

                # clamp
                x1 = max(0, x1)
                y1 = max(0, y1)
                x2 = min(w, x1 + bw)
                y2 = min(h, y1 + bh)

                face = frame[y1:y2, x1:x2]

                if face.size > 0:
                    face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
                    out_name = f"{video_name}_{saved:03d}.jpg"
                    cv2.imwrite(os.path.join(LIVE_OUT, out_name), face)

                    saved += 1
                    img_count += 1

        frame_id += 1
        if saved >= MAX_FRAMES_PER_VIDEO:
            break

    cap.release()

print(f"✅ Done. Saved {img_count} live face images to {LIVE_OUT}")


🔍 Found 30 live videos
✅ Done. Saved 596 live face images to WEBCAM\live


In [5]:
import os
import shutil

# =========================
# CONFIG
# =========================
ROOT_DATASET = "dataset-2"
SAMPLES_DIR = os.path.join(ROOT_DATASET, "samples")

VIDEO_PHONE_DIR = "LIVE_VIDEOS_FOR_PHONE"
os.makedirs(VIDEO_PHONE_DIR, exist_ok=True)

video_count = 0

# =========================
# COPY VIDEOS
# =========================
for sample_id in os.listdir(SAMPLES_DIR):
    sample_path = os.path.join(SAMPLES_DIR, sample_id)
    if not os.path.isdir(sample_path):
        continue

    for f in os.listdir(sample_path):
        if f.lower().startswith("live_video"):
            src = os.path.join(sample_path, f)
            dst = os.path.join(VIDEO_PHONE_DIR, f"{sample_id}.mp4")
            shutil.copy(src, dst)
            video_count += 1
            break

print("✅ VIDEO COPY DONE")
print(f"🎥 Total videos copied: {video_count}")
print(f"📂 Folder for phone: {VIDEO_PHONE_DIR}")


✅ VIDEO COPY DONE
🎥 Total videos copied: 30
📂 Folder for phone: LIVE_VIDEOS_FOR_PHONE


In [2]:
import os
import cv2
import numpy as np
from mtcnn import MTCNN

# =========================
# CONFIG
# =========================
SPOOF_VIDEO_DIR = "SPOOF_VIDEO"
OUTPUT_DIR = "WEBCAM/not_live"

IMG_SIZE = 256
TARGET_IMAGES = 1800     # 🔥 số ảnh spoof mong muốn
FRAME_INTERVAL = 6       # lấy dày hơn live một chút (spoof cần đa dạng)

os.makedirs(OUTPUT_DIR, exist_ok=True)

detector = MTCNN()
img_count = 0

video_list = [
    os.path.join(SPOOF_VIDEO_DIR, f)
    for f in os.listdir(SPOOF_VIDEO_DIR)
    if f.lower().endswith((".mp4", ".mov", ".avi"))
]

print(f"🎥 Found {len(video_list)} spoof videos")

# =========================
# PROCESS VIDEOS
# =========================
for video_path in video_list:
    if img_count >= TARGET_IMAGES:
        break

    cap = cv2.VideoCapture(video_path)
    frame_idx = 0

    while cap.isOpened():
        if img_count >= TARGET_IMAGES:
            break

        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % FRAME_INTERVAL != 0:
            frame_idx += 1
            continue

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        faces = detector.detect_faces(rgb)

        if len(faces) > 0:
            # lấy khuôn mặt lớn nhất
            faces = sorted(
                faces,
                key=lambda x: x["box"][2] * x["box"][3],
                reverse=True
            )

            x, y, w, h = faces[0]["box"]
            x, y = max(0, x), max(0, y)

            # 🔹 thêm margin để giữ viền màn hình spoof
            margin = int(0.15 * max(w, h))
            x1 = max(0, x - margin)
            y1 = max(0, y - margin)
            x2 = min(frame.shape[1], x + w + margin)
            y2 = min(frame.shape[0], y + h + margin)

            face_crop = frame[y1:y2, x1:x2]
            if face_crop.size == 0:
                frame_idx += 1
                continue

            face_crop = cv2.resize(face_crop, (IMG_SIZE, IMG_SIZE))

            img_name = f"spoof_{img_count:05d}.jpg"
            cv2.imwrite(os.path.join(OUTPUT_DIR, img_name), face_crop)
            img_count += 1

        frame_idx += 1

    cap.release()

print("✅ SPOOF EXTRACTION DONE")
print(f"📸 Total spoof images: {img_count}")
print(f"📂 Output folder: {OUTPUT_DIR}")


🎥 Found 31 spoof videos
✅ SPOOF EXTRACTION DONE
📸 Total spoof images: 1800
📂 Output folder: WEBCAM/not_live
